In [1]:
import os
import sys
import platform
import subprocess
from pathlib import Path
from GraphTsetlinMachine.graphs import Graphs
from GraphTsetlinMachine.tm import MultiClassGraphTsetlinMachine
import pickle
import json
import logging
import optuna
from optuna.exceptions import TrialPruned 
from tqdm.auto import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score
import math
from functools import partial
import warnings
import random
from src.dbhandler import DBHandler
import time

In [2]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [3]:
AWS_DB = True
STUDY_NAME = "Hex_hv_acc_only"
windows_drive = Path("/mnt/f/TsetlinModels")

In [4]:
if AWS_DB:
    db = DBHandler()
    DB = db.target_storage_url

In [5]:
def get_machine_info():
    machine_name = platform.node()  
    user = os.getenv("USER") or os.getenv("USERNAME") 
    os_name = platform.system()  # Get os
    print(f"Machine: {machine_name}")
    print(f"OS: {os_name}")
    print(f"User: {user}")
    
    # Print machine info
    return machine_name, os_name, user

In [6]:
def get_score_v1(result:float, board_size:int, number_of_clauses:int):
    return result * board_size - 1e-4 * number_of_clauses

In [7]:
def objective(trial, paths):

    #start = time.time()
    

    board_size = trial.suggest_int('board_size', 4, 15) 
    
    n_samples_map = {0: 1000, 1: 10000, 2: 100000}
    n_samples_idx = trial.suggest_int('n_samples_idx', 0, 2)  
    n_samples = n_samples_map[n_samples_idx]  
    
    mbf_map = {0: 0, 1: 2, 2: 5}
    mbf_idx = trial.suggest_int('mbf_idx', 0, 2)  
    mbf = mbf_map[mbf_idx]  
    
    pos = trial.suggest_int('open_pos', 0, 50, step=5)  

    message_size_map = {0: 32, 1: 64, 2: 128, 3: 512, 4: 1024, 5: 2048, 6: 4096, 7: 8192, 8: 16384, 9: 32768} # Skipped 256 because of unknown error
    message_size_idx = trial.suggest_int('message_size_idx', 0, 9)
    message_size = message_size_map[message_size_idx]

    message_bits_map = {0: 1, 1: 2, 2: 5, 3: 10, 4: 15, 5: 20, 6: 30, 7: 50}
    hv_mb_ratio_idx = trial.suggest_int('hv_mb_ratio_idx', 0, 2)#7) 
    hv_mb_ratio = message_bits_map[hv_mb_ratio_idx]
    message_bits = max(1, int(hv_mb_ratio * message_size * 0.01))

    dh_map = {0: False, 1: True}
    dh_idx = trial.suggest_int('dh_idx', 0, 1)
    dh = dh_map[dh_idx]
    dh_string = "dh2" if dh else "dh1"

    # Load the corresponding dataset based on trial's suggestions
    dataset = f"{board_size}x{board_size}_{n_samples}_{pos}_{mbf}_{message_size}_{hv_mb_ratio}_{dh_string}"
    file_path = paths["graphs"] / f"{dataset}.pkl"
    with open(file_path, 'rb') as f:
        graphs_train, graphs_test, X_train, Y_train, X_test, Y_test = pickle.load(f)

    trial_num = trial.number

    number_of_clauses = trial.suggest_int('number_of_clauses', 2, 10000)
    T = trial.suggest_float('T_factor', 0.2, 1.5) * number_of_clauses
    s = trial.suggest_float('s', 0, 10)
    q = trial.suggest_float('q', 0.4, 1.0)
    
    max_included_literals = trial.suggest_int('max_included_literals', 10, 300)
    if max_included_literals == 300:
        max_included_literals = None

    depth = trial.suggest_int('depth', 1, board_size*2)
    number_of_state_bits = trial.suggest_int('number_of_state_bits', 8, 1000)
    epochs = trial.suggest_int('epochs', 10, 80)

    tm = MultiClassGraphTsetlinMachine(
        number_of_clauses,
        T,
        s,
        q=q,
		max_included_literals=max_included_literals,
        depth=depth,
        message_size=message_size,
        message_bits=message_bits,
        number_of_state_bits=number_of_state_bits,
        boost_true_positive_feedback=1,
        grid=(16*13,1,1),
        block=(128,1,1),
    )

    
    print(f"c={number_of_clauses}, T={T:.4f}, s={s:.4f}, q={q:.4f}, d={depth}, e={epochs}, ms={message_size}, mb={message_bits}, mil={max_included_literals}, sb={number_of_state_bits}, dh={dh}")

    best_test_acc = 0
    best_f1 = 0
    best_prec = 0
    best_rec = 0
    patience = 0
    patience_level = epochs*0.25
    result_train = 0
    overfit = 0
    score = 0
    
    progress_bar = tqdm(range(epochs), desc=f"{dataset} [{trial_num}]", leave=True)
    for epoch in progress_bar:
        tm.fit(graphs_train, Y_train, epochs=1, incremental=True)

        result_test = 100 * (tm.predict(graphs_test) == Y_test).mean()

        if epoch % 10 == 0:
            result_train = 100 * (tm.predict(graphs_train) == Y_train).mean() 
            
        #overfit = abs(result_train - result_test)
            
            

        if result_test > best_test_acc:
            best_test_acc = result_test
            f1_score_test = f1_score(Y_test, tm.predict(graphs_test), average='weighted', zero_division=0)
            precision_test = precision_score(Y_test, tm.predict(graphs_test), average='weighted', zero_division=0)
            recall_test = recall_score(Y_test, tm.predict(graphs_test), average='weighted', zero_division=0)
            best_f1 = f1_score_test
            best_prec = precision_test
            best_rec = recall_test
            trial.set_user_attr("f1", f1_score_test)
            trial.set_user_attr("precision", precision_test)
            trial.set_user_attr("recall", recall_test)
            patience = 0
        else:
            patience += 1

        trial.set_user_attr("acc_train", result_train)
        trial.set_user_attr("acc_test", result_test)
        trial.set_user_attr("score_func", "v1")

        
        progress_bar.set_postfix({
            'X-Acc':f'{result_train:.2f}%',
            'Y-Acc':f'{result_test:.2f}%',
            'BestAcc': f'{best_test_acc:.2f}%',
            'F1': f'{best_f1:.2f}',
            'Prec': f'{best_prec:.2f}',
            'Rec': f'{best_rec:.2f}'
        })

        
        if result_test >= 100 and f1_score_test >= 1:
            result_train = 100 * (tm.predict(graphs_train) == Y_train).mean()
            trial.set_user_attr("acc_train", result_train)
            progress_bar.set_postfix({
                'X-Acc':f'{result_train:.2f}%',
                'Y-Acc':f'{result_test:.2f}%',
                'BestAcc': f'{best_test_acc:.2f}%',
                'F1': f'{best_f1:.2f}',
                'Prec': f'{best_prec:.2f}',
                'Rec': f'{best_rec:.2f}'
            })
            #end = time.time()
            #total_time = end - start

            return best_test_acc#, overfit, number_of_clauses, total_time

            
            
        trial.report(get_score_v1(result_test, board_size, number_of_clauses), epoch)

        if patience >= patience_level and best_test_acc < 85:
            if trial.should_prune():
                raise TrialPruned()
    
    #end = time.time()
    #total_time = end - start
    

    return best_test_acc#, overfit, number_of_clauses, total_time

In [8]:

machine_name, os_name, user = get_machine_info()

if machine_name == "Corsair" and os_name == "Linux" and user == "jon":
    os.makedirs(windows_drive / "data", exist_ok=True)
    os.makedirs(windows_drive / "models", exist_ok=True)
    os.makedirs(windows_drive / "graphs", exist_ok=True)
    os.makedirs(windows_drive / "studies", exist_ok=True)

    paths = {
        "data": windows_drive / "data",
        "models": windows_drive / "models",
        "graphs": windows_drive / "graphs",
        "studies": windows_drive / "studies",
    }

    if not AWS_DB:
        DB = f"sqlite:///{windows_drive / 'studies' / f'{STUDY_NAME}.db'}" 

else:
    os.makedirs("data", exist_ok=True)
    os.makedirs("models", exist_ok=True)
    os.makedirs("graphs", exist_ok=True)

    paths = {
        "data": Path("data"),
        "models": Path("models"),
        "graphs": Path("graphs"),
    }
    if not AWS_DB:
        DB = f"sqlite:///results/optuna/{STUDY_NAME}.db"

Machine: Corsair
OS: Linux
User: jon


In [9]:
study = optuna.create_study(
    directions=["maximize"],#"minimize", "minimize", "minimize"],
    study_name=STUDY_NAME,
    storage=str(DB),
    load_if_exists=True,
)

In [10]:
def read_board_size(board_size_file):
    if os.path.exists(board_size_file):
        with open(board_size_file, "r") as f:
            data = json.load(f)
            return data.get("board_size", 4) 
    return 4 


def stop_on_max_accuracy(study, trial):
    if trial.user_attrs.get("acc_test", 0) >= 100.0:
        print(f"Reached {100.0}% accuracy with board_size={trial.params['board_size']}")
        study.stop() 

def write_board_size(filename, board_size):
    import json
    with open(filename, "w") as f:
        json.dump({"board_size": board_size}, f)

In [11]:
optuna_port = 8083
storage = DB
optuna_process = subprocess.Popen(
    ["optuna-dashboard", "-q", "--port", str(optuna_port), storage],
    stdout=subprocess.DEVNULL,  # Suppresses standard output
    stderr=subprocess.DEVNULL   # Suppresses standard error (warnings and logs)
)

print(f"Optuna Dashboard is running. Access it via http://localhost:{optuna_port}/")

Optuna Dashboard is running. Access it via http://localhost:8083/


In [12]:
warnings.filterwarnings("ignore", category=optuna.exceptions.ExperimentalWarning)

board_size_file = "board_size.json"
max_accuracy_threshold = 100.0
initial_board_size = read_board_size(board_size_file)

In [ ]:
parameter_options = {
    "n_samples_idx": [0, 1, 2],
    "mbf_idx": [0, 1, 2],
    "open_pos": [0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50],
    "number_of_clauses": [100,200,500,800,1000],
    "s": [0.001, 0.01, 0.1],
    "depth": [1, 2, 3],
    "message_size_idx": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
    "hv_mb_ratio_idx": [0, 1, 2, 3, 4, 5, 6, 7],
    "number_of_state_bits": [10, 20, 40, 50, 100, 200, 500, 1000],
}


while True:
    fixed_params = {
        "board_size": 4,#initial_board_size,
        "n_samples_idx": 0,         # 1000,10000,100000
        "mbf_idx": 0,               # 0,2,5
        #"open_pos": 50,             # 0,5,10,15,20,25,30,35,40,45,50
        "number_of_clauses": initial_board_size**2,
        #"T_factor": 1,
        #"s": 0.001,
        #"q": 0.9,
        "max_included_literals": 10,
        "depth": 2,#initial_board_size,
        "message_size_idx": 0,      # 32,64,128,512,1024,2048,4096,8192,16384,32768
        "hv_mb_ratio_idx": 0,       # 1,2,5,10,15,20,30,50
        "number_of_state_bits": initial_board_size**2, 
        }

    if initial_board_size <= 5: 
        fixed_params["n_samples_idx"] = 0

    # Decide randomly whether to freeze a parameter
    if random.choice([True, False]): 
        param_to_freeze = random.choice(list(parameter_options.keys()))
        if param_to_freeze not in fixed_params:
            frozen_value = random.choice(parameter_options[param_to_freeze])
            fixed_params[param_to_freeze] = frozen_value
            print(f"Freezing parameter: {param_to_freeze} = {frozen_value}")
    
    sampler = optuna.samplers.PartialFixedSampler(fixed_params, optuna.samplers.TPESampler())
    study.sampler = sampler

    try:
        study.optimize(
            lambda trial: objective(trial, paths),
            n_trials=2,
            callbacks=[stop_on_max_accuracy] 
        )
    except KeyboardInterrupt:
        print("Optimization interrupted!")
        break
        
    current_board_trials = [
        trial for trial in study.trials
        if trial.params.get("board_size") == initial_board_size
    ]

    # Get the maximum accuracy for this board size
    max_accuracy_for_board = max(
        (trial.user_attrs.get("acc_test", 0) for trial in current_board_trials),
        default=0
    )
    

    # Check if the maximum accuracy for the current board size reached the threshold
    if max_accuracy_for_board >= max_accuracy_threshold:
        print(f"✅ Achieved 100% accuracy for board_size={initial_board_size}")
        initial_board_size += 1  # Only increase the board size when 100% is achieved
        write_board_size(board_size_file, initial_board_size)
        print(f"🆙🆙🆙 Increasing board_size to {initial_board_size} 🆙🆙🆙")
    else:
        print(f"Max accuracy for board_size {initial_board_size}x{initial_board_size} is {max_accuracy_for_board:.2f}%. Let's try some more...")
        

print("Optimization finished!")
optuna_process.terminate()

Initialization of sparse structure.
c=36, T=35.6626, s=8.1287, q=0.8904, d=2, e=78, ms=32, mb=1, mil=10, sb=36, dh=True


4x4_1000_30_0_32_1_dh2 [0]:   0%|          | 0/78 [00:00<?, ?it/s]

Initialization of sparse structure.
c=36, T=51.5763, s=3.5454, q=0.6372, d=2, e=14, ms=32, mb=1, mil=10, sb=36, dh=True


4x4_1000_10_0_32_1_dh2 [1]:   0%|          | 0/14 [00:00<?, ?it/s]

Max accuracy for board_size 6x6 is 0.00%. Let's try some more...
Initialization of sparse structure.
c=36, T=48.5785, s=1.3484, q=0.8891, d=2, e=55, ms=32, mb=1, mil=10, sb=36, dh=False


4x4_1000_5_0_32_1_dh1 [2]:   0%|          | 0/55 [00:00<?, ?it/s]